# TfidfVectorizer Nedir?

### TF-IDF = Term Frequency * Inverse Document Frequency


- TF(Term Frequency): Bir kelimenin dökümanda ne kadar sık geçtiğidir

- IDF(Inverse Document Frequency): Bir kelimenin tüm koleksiyonda(tüm dokumanların toplamı,analiz ettiğimiz veri seti) ne kadar nadir olduğudur

- DF(Document Frequency): Bir kelimenin tüm koleksiyonda ne kadar geçtiği

### Neden Kullanılır?

1-) Makine öğrenmesi algoritmaları sayılarla çalışır. Metini sayıya çevirmeliyiz

2-)  Önemli kelimeleri vurgular. Nadir ama anlamlı kelimeler yüksek puan(ağırlık) alır

3-) Anlamsız kelimeleri bastırır. the,and,is gibi kelimeler üdşük sor alır(CountVectorizer'dan farkı)

### Temel Adımlar

1-) fit_transform() : İlk veri seti için(öğren-dönüştür)

2-) transform() : Yeni veriler için (sadece dönüştür)

3-) Makine öğrenmesi modeli ile tahmin yapma

##  Kod Örneği

- Kod örneğinde e-posta spam tespiti yaparak gerçek hayata uygun bir problem çözecez

In [116]:
# Bu örnek e-postaya gelen metinleri analiz ederek spam/ham(normal) sınıflandırması yapar

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

""" 
MultinomialNB algoritması kullanma sebebi metin verileri için uygun olmasıdır
Sayma verisi(count data) ile çalışır
Sıfır değerler ile rahatlıkla çalışır(bazı kelimeler dokümanda geçmez)
TF-IDF skorları multinominal dağılıma uygundur
"""

' \nMultinomialNB algoritması kullanma sebebi metin verileri için uygun olmasıdır\nSayma verisi(count data) ile çalışır\nSıfır değerler ile rahatlıkla çalışır(bazı kelimeler dokümanda geçmez)\nTF-IDF skorları multinominal dağılıma uygundur\n'

In [117]:
# Örnek e-posta veri seti oluştur
emails=[
    #SPAM e-postalar
    "Congratulations! You won $1000000! Click here to claim your prize now!",
    "URGENT: Your account will be closed. Send your password immediately.",
    "Free money! Win big! Casino online! Click now for amazing offers!",
    "Viagra cheap! Best prices! Order now! Special discount today only!",
    "You are selected winner! Claim $50000 prize money today! Act fast!",
    "Make money fast! Work from home! Earn $5000 weekly! No experience needed!",
    "FREE! FREE! FREE! Download now! Limited time offer! Click here!",
    "Your credit approved! $10000 loan instant approval! Apply now!",

    # HAM(normal) epostalar

    "Hi Trump, can we schedule a meeting for tomorrow afternoon?",
    "The quarterly report is ready for review. Please check the attached file.",
    "Thank you for your presentation today. It was very informative.",
    "Reminder: Team lunch is scheduled for Friday at 12:30 PM.",
    "Please find the updated project timeline in the attachment.",
    "Good morning! How was your weekend? Looking forward to our call.",
    "The software update has been completed successfully on all systems.",
    "Your order has been shipped and will arrive within 3-5 business days."

]

#Etiketler (0: Ham, 1: SPAM)
labels=[1,1,1,1,1,1,1,1,#İLk 8 SPAM
        0,0,0,0,0,0,0,0] # Son 8 Ham

"""
Formül
TF-IDF(t,d) = (t sayısı / toplam kelime sayısı) * log(toplam döküman / t içeren döküman sayısı)
Amaç: Önemli ama nadir bulunan kelimelere yüksek puan vermek
"""

'\nFormül\nTF-IDF(t,d) = (t sayısı / toplam kelime sayısı) * log(toplam döküman / t içeren döküman sayısı)\nAmaç: Önemli ama nadir bulunan kelimelere yüksek puan vermek\n'

In [118]:
# TfidfVectorizer oluştur ve parametreleri açıkla

vectorizer=TfidfVectorizer(
    max_features=60 ,  #En fazla 60 özellik(kelime) kullan
    stop_words="english",   #İngilizce stop words'leri kaldır(a,an,the,is vs) kaldır
    lowercase=True,   #Tüm harfleri küçük harfe çevir
    ngram_range=(1,2)   #Tek kelimeler(1-gram) ve kelime çiftleri (2-gram) kullan
)

In [119]:
# Metinleri vektöre çevir

#fit_transform ile hem öğren hem dönüştür
X=vectorizer.fit_transform(emails)
print(f"Orijinal metin sayısı: {len(emails)}")
print(f"Oluşturan vektör boyutu: {X.shape}")
print(f"Her metin {X.shape[1]} boyutlu bir vektör haline geldi")

#Hangi kelimeler kullanıldı
feature_names=vectorizer.get_feature_names_out()
print(f"Kullanılan kelimelerin ilk 20 tanesi {feature_names[:20]}")

Orijinal metin sayısı: 16
Oluşturan vektör boyutu: (16, 60)
Her metin 60 boyutlu bir vektör haline geldi
Kullanılan kelimelerin ilk 20 tanesi ['10000' 'claim' 'click' 'fast' 'free' 'free free' 'meeting tomorrow'
 'money' 'money fast' 'money today' 'money win' 'morning'
 'morning weekend' 'needed' 'offer' 'offer click' 'offers' 'online'
 'online click' 'order']


In [120]:
# Bir tane örnek metnin nasıl vektöre çevrildiğini görelim
print("ÖRNEK METİN ANALİZİ")
sample_text=emails[0]#ilk spam maili
sample_vector=X[0].toarray()[0] #bu metnin vektör hali
print(f"Örnek metin: {sample_text}")
print(f"Bu metin {len(sample_vector)} boyutlu vektöre dönüştü")

# Örnek metindeki en önemli(en yüksek skorlu) 10 kelimeyi gösteer
top_indices=sample_vector.argsort()[-10:][::-1]
print("Bu metindeli en önemli 10 kelime ve TF-IDF skorları")
for idx in top_indices:
    if sample_vector[idx]>0:
        print(f"{feature_names[idx]} : {sample_vector[idx]}")

ÖRNEK METİN ANALİZİ
Örnek metin: Congratulations! You won $1000000! Click here to claim your prize now!
Bu metin 60 boyutlu vektöre dönüştü
Bu metindeli en önemli 10 kelime ve TF-IDF skorları
prize : 0.5975432755013906
claim : 0.5975432755013906
click : 0.5346812768426984


In [121]:
# Makine öğrenmesi modeli oluştur
X_train,X_test,y_train,y_test=train_test_split(X,labels,stratify=labels,test_size=0.33,random_state=42)
print(f"Eğitim seti: {X_train.shape[0]} örnekten oluşuyor")
print(f"Test seti {X_test.shape[0]} örnekten oluşuyor")

#Naive bayes sınıflandırıcısı ile modeli eğit
model=MultinomialNB()
model.fit(X_train,y_train)

#Test seti üzerinden tahmin yap
y_pred=model.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)

print(f"Model doğruluğu : {accuracy:.2f}")

Eğitim seti: 10 örnekten oluşuyor
Test seti 6 örnekten oluşuyor
Model doğruluğu : 0.83


In [122]:
# Sonuçları analiz et
print("Detaylı analiz")
print(classification_report(y_test,y_pred,target_names=["HAM","SPAM"]))

print("--"*31)
#Karışıklık matrisi
cm=confusion_matrix(y_test,y_pred)
print("Karışıklık Matrisi")
cm

Detaylı analiz
              precision    recall  f1-score   support

         HAM       0.75      1.00      0.86         3
        SPAM       1.00      0.67      0.80         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6

--------------------------------------------------------------
Karışıklık Matrisi


array([[3, 0],
       [1, 2]])

In [123]:
# Yeni e-posta örnekleri ile test et
new_emails=["Contact me",
            "WIN BIG MONEY NOW! CLICK HERE FOT INSTANT CASH",
            "The meeting has been reschueduled to 3 Pm tomorrow"]

# Yeni mailleri aynı vektorizastyon ile dönüştür
# fit_transform değil sadece transform
new_X=vectorizer.transform(new_emails)
new_predictions=model.predict(new_X)
new_probabalities=model.predict_proba(new_X)

for i,email in enumerate(new_emails):
    prediction= "SPAM" if new_predictions[i]== 1 else "HAM"
    spam_prob=new_probabalities[i][1] * 100
    print(f"E-Posta {email}")
    print(f"Tahmin: {prediction} (Spam olma olasılığı: {spam_prob:.2f})")

E-Posta Contact me
Tahmin: HAM (Spam olma olasılığı: 50.00)
E-Posta WIN BIG MONEY NOW! CLICK HERE FOT INSTANT CASH
Tahmin: SPAM (Spam olma olasılığı: 66.40)
E-Posta The meeting has been reschueduled to 3 Pm tomorrow
Tahmin: HAM (Spam olma olasılığı: 49.83)


In [124]:
# En önemli kelimeleri bul
feature_importance=model.feature_log_prob_[1] - model.feature_log_prob_[0]
top_spam_indices=feature_importance.argsort()[-10:][::1]

print("SPAM tespiti için en etkili kelimeler")
for idx in top_indices:
    print(f"{feature_names[idx]}: {feature_importance[idx]:.4f}")

SPAM tespiti için en etkili kelimeler
prize: 0.6555
claim: 0.6555
click: 0.4215
offer click: -0.0068
prices: 0.3270
presentation today: -0.4888
presentation: -0.4888
pm: -0.0068
password immediately: -0.0068
password: -0.0068


In [ ]:
#zgncn